In [12]:
from PIL import Image, ImageDraw
import os
import collections
from math import floor
import sys

def direct(name_file):
    path=os.getcwd()
    path+='/ignis'
    path+='/'+name_file.split('.')[0]
    try:
        os.mkdir(path) # только на один уровень ниже. иначе исп. makedirs()
    except OSError:
        print('Создать дирректорию %s не удалось. Возможно, она уже была создана.' % path)
    else:
        print('Успешно создана дирректория %s' % path)
    return path

def ren32(path_pic,path,num,metka):
    try:
        image=Image.open(path_pic+metka+'.bmp')
        image.save(path+'/'+'{:0>2}_{}.bmp'.format(num,metka), "BMP")
    except FileNotFoundError:
        print()
        print('НЕТ ФАЙЛА %s \n' %(path_pic+metka+'.bmp'))
        
def ren64(path_pic,path,num,metka):
    image1 = Image.open(path_pic+metka+'.bmp') #Открываем изображение. 
    image2=Image.open(path_pic+'white_64.bmp')
    width = image1.size[0] #Определяем ширину. 
    height1 = image1.size[1] #Определяем высоту.

    height2 = image2.size[1] #Определяем высоту.
    draw = ImageDraw.Draw(image2) #Создаем инструмент для рисования. 
    pix = image1.load()

    for i in range(width):
        k=-1
        for j in range(height1):
            a = pix[i, j][0]
            b = pix[i, j][1]
            c = pix[i, j][2]
            k+=1
            draw.point((i, k), (a, b, c))
            k+=1
            draw.point((i, k), (a, b, c))

    image2.save(path+'/'+'{:0>2}_{}.bmp'.format(num,metka), "BMP")
    del draw
#print('ВВедите имя будущей папки для реквизита. Помните, что оно должно начинаться с  XX_prog_ 
#print('где XX - порядковый  номер программы у пойки')

#########name_file_out=input()  ################################
path_pic='/home/gatupov/python/ignis/pictures/' ## ОБЯЗАТЕЛЬНО ПОМЕНЯТЬ ПОЛЬЗОВАТЕЛЮ!!!!!!
#######################################################
name_file_out='04_prog_routable_poi'
name_path=direct(name_file_out)
file_out=open(name_path+'/program.txt','w')

time=[]
metka=[]
person=[]
#print('Введите имя дорожки отметок')
#name_file=input()
rekv=[]
person_rekv=[]
flag_rekv=0
name_file='routable_poi_metki.txt'
with open(name_file) as mfile:
    for st in mfile: # проверка
        if not(len(st.split('\t'))==3) and not('#' in st):
            print('DANGER')
        else:
            if ('#' in st):
                line=st.split('-')[1]
                line=line.replace(' ','')
                line=line.replace('\t','')
                line=line.replace('\r','')
                line=line.replace('\n','')
                rekv.append(int(line)) 
                
                line=st.split('-')[0]
                line=line.replace('#','')
                line=line.replace(' ','')
                line=line.replace('\t','')
                line=line.replace('\r','')
                line=line.replace('\n','')
                person_rekv.append(line)
                flag_rekv+=1
            if not('#' in st):   
                time.append(st.split('\t')[1]) 
                s=st.split('\t')[2]
                if '>' in s:
                    if '/' in s:
                        chasti=s.split('/')
                        person_sbor=''
                        metka_sbor=''
                        for ch in chasti:
                            person_sbor+=ch.split('>')[0]+';'
                            metka_sbor+=ch.split('>')[1]+';'
                        person.append(person_sbor[:-1])
                        metka.append(metka_sbor[:-2])
                    else:
                        person.append(s.split('>')[0])
                        metka.append(s.split('>')[1][:-1])
                else:
                    person.append('all')
                    metka.append(s[:-1])
print()
print(metka)
print(time)
print(person_rekv)
print(rekv)


all_persons=[]
for elem in person:
    pers=elem.split(';')
    for p in pers:
        all_persons.append(p)

persons=list(collections.Counter(all_persons).keys())
rekv2=[int(32)]*len(persons)

if flag_rekv>1:
    for j in range(len(persons)):
        for i in range(len(person_rekv)):
            if person_rekv[i]==persons[j]:
                rekv2[j]=rekv[i]
if flag_rekv==1:
    for j in range(len(persons)):
        rekv2[j]=rekv[0]


            
print(persons)
people=0

if len(persons)==1 and persons[0]=='all':
    people=1 # одна прога на всех
    if flag_rekv==0:
        print("Ошибка: вы не указали длину реквизита в пикселях в начале документа")
        sys.exit()

dtime=[]
stime=[]
pop=0
for j in range(len(time)):
    dtime.append(( float(time[j]) - float(time[0]) ))
for j in range(len(time)):
    minn=int(dtime[j]//60)
    sec=int(floor(dtime[j]%60))-pop
    if (sec==-1):
        sec=59
        minn-=1
    msec=int(floor((dtime[j] - floor(dtime[j]%60) - minn*60)*100))
    stime.append('{:0>2}:{:0>2}:{:0>2}'.format(minn,sec,msec))
    pop=1
    
print(stime)
#dop='Finish - {}\nRepeat after finish - no\nLock buttons - yes'.format(stime[len(time)-1])

if people==1:
    if flag_rekv==0:
        print("Ошибка: вы не указали длину реквизита в пикселях в начале документа")
        sys.exit()
    
    numer=0
    for j in range(len(time)):
        if metka[j]=='end':
            dop='Finish - {}\nRepeat after finish - no\nLock buttons - yes'.format(stime[j])
            file_out.write(dop)
        else:
            numer+=1
            if rekv[0]==32:
                ren32(path_pic,name_path,numer,metka[j])
            if rekv[0]==64:
                ren64(path_pic,name_path,numer,metka[j])
            if not(rekv[0]==32) and not(rekv[0]==64):
                print('Описка в указании длины реквизита. Неверное число пикселей')
                print('Допустимое число пикселей - 32 или 64')
            file_out.write('{:0>2}_{} - {}\n'.format(numer,metka[j],stime[j]))
    
    
    #file_out.write(dop)
    file_out.close()
    print('Записано в '+ name_file_out)

if people==0:
    person1=[]
    rekv3=[]
    for t in range(len(persons)):
        if not persons[t]=='all':
            person1.append(persons[t])
            rekv3.append(rekv2[t])
    persons=person1
    rekv=rekv3
    file_out.write('Ищи в других файлах с номерами')
    file_out.close()
    per_n=-1
    for per in persons:
        per_n+=1
        numer=0
        name_path=direct(name_file_out+'_'+per)
        file_out=open(name_path+'/program.txt','w')
        for j in range(len(time)):
            if metka[j]=='end':
                dop='Finish - {}\nRepeat after finish - no\nLock buttons - yes'.format(stime[j])
                file_out.write(dop)
            else:
                flag_m=0
                kombo_metka=metka[j].split(';')
                kombo_person=person[j].split(';')
                for m in range(len(kombo_metka)):

                    if per==kombo_person[m] or person[j]=='all':
                        numer+=1
                        if rekv[per_n]==32:
                            ren32(path_pic,name_path,numer,kombo_metka[m])
                        if rekv[per_n]==64:
                            ren64(path_pic,name_path,numer,kombo_metka[m])
                        if not(rekv[per_n]==32) and not(rekv[per_n]==64):
                            print('Описка в указании длины реквизита. Неверное число пикселей')
                            print('Допустимое число пикселей - 32 или 64')
                        file_out.write('{:0>2}_{} - {}\n'.format(numer,kombo_metka[m],stime[j]))
                        flag_m=1
                if flag_m==0:
                    numer+=1
                    if rekv[per_n]==32:
                        ren32(path_pic,name_path,numer,'blackout')
                    if rekv[per_n]==64:
                        ren64(path_pic,name_path,numer,'blackout')
                    if not(rekv[per_n]==32) and not(rekv[per_n]==64):
                        print('Описка в указании длины реквизита. Неверное число пикселей')
                        print('Допустимое число пикселей - 32 или 64')
                    file_out.write('{:0>2}_{} - {}\n'.format(numer,'blackout',stime[j]))
        file_out.close()
        print('Записано в '+name_file_out+'_'+per)
        

Создать дирректорию /home/gatupov/python/ignis/04_prog_routable_poi не удалось. Возможно, она уже была создана.

['blackout', 'red', 'yellow', 'blue', 'green', 'blue;blue', 'red;red', 'galka;galka', 'violet', 'red', 'purple', 'end']
['3.991117', '6.482285', '10.230897', '13.980518', '17.731085', '19.625011', '21.423235', '29.072371', '34.615926', '36.481274', '43.989096', '51.478749']
['1d', '2d', '3d', '4d']
[32, 32, 64, 64]
['all', '1d', '2d', '3d', '4d']
['00:00:00', '00:01:49', '00:05:23', '00:08:98', '00:12:73', '00:14:63', '00:16:43', '00:24:08', '00:29:62', '00:31:49', '00:38:99', '00:46:48']
Создать дирректорию /home/gatupov/python/ignis/04_prog_routable_poi_1d не удалось. Возможно, она уже была создана.
Записано в 04_prog_routable_poi_1d
Создать дирректорию /home/gatupov/python/ignis/04_prog_routable_poi_2d не удалось. Возможно, она уже была создана.
Записано в 04_prog_routable_poi_2d
Создать дирректорию /home/gatupov/python/ignis/04_prog_routable_poi_3d не удалось. Возможно, 

In [1]:
print(floor(5.3))

NameError: name 'floor' is not defined